In [20]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from konlpy.tag import Okt   

In [43]:
res_edu = pd.read_csv("resume_education.csv")
eduGrouped = res_edu.sort_values('resume_seq')
len(eduGrouped['resume_seq'].unique()) 
# 학벌은 중복이 되지 않음.
# 고등학교 타입 12
# 학교 특징. 5개
# 공립사립기타 3개
# 남/여 여부
# 학교위치 17
# 대학타입 6개
# 대학타입2 10개
# 대학위치 18개
# 대학 전공타입 23개
# 점수는 96. 연속형 값이라 정규화 가능.
# 대학편입요소(transfer)는 중요하다고 판단하지 않음

8482

In [44]:
# 학벌은 중복이 되지 않음.
eduGrouped =eduGrouped.set_index(['resume_seq'])

In [45]:
eduGrouped['hischool_type_seq']=eduGrouped['hischool_type_seq'].astype('category')               
eduGrouped['hischool_special_type']=eduGrouped['hischool_special_type'].astype('category')    
eduGrouped['hischool_nation']=eduGrouped['hischool_nation'].astype('category')    
eduGrouped['hischool_gender']=eduGrouped['hischool_gender'].astype('category')
eduGrouped['univ_type_seq1']=eduGrouped['univ_type_seq1'].astype('category')
eduGrouped['univ_type_seq2']=eduGrouped['univ_type_seq2'].astype('category')
eduGrouped['univ_location']=eduGrouped['univ_location'].astype('category')
eduGrouped['univ_major_type']=eduGrouped['univ_major_type'].astype('category')

In [46]:
add1 = pd.get_dummies(eduGrouped['hischool_type_seq'],prefix ='hischool_type',dtype='int') 
add2 = pd.get_dummies(eduGrouped['hischool_special_type'],prefix ='hischool_special',dtype='int') 
add3 = pd.get_dummies(eduGrouped['hischool_nation'],prefix ='hischool_nation',dtype='int') 
add4 = pd.get_dummies(eduGrouped['hischool_gender'],prefix ='hischool_gender',dtype='int') 
add5 = pd.get_dummies(eduGrouped['univ_type_seq1'],prefix ='univ_type1',dtype='int') 
add6 = pd.get_dummies(eduGrouped['univ_type_seq2'],prefix ='univ_type2',dtype='int') 
add7 = pd.get_dummies(eduGrouped['univ_location'],prefix ='univ_location',dtype='int') 
add8 = pd.get_dummies(eduGrouped['univ_major_type'],prefix ='major_type',dtype='int') 
eduGrouped = pd.concat([eduGrouped,add1,add2,add3,add4,add5,add6,add7,add8],axis=1)

In [47]:
scaler = MinMaxScaler()
data = eduGrouped['univ_score'].values.reshape(-1, 1)
data2= pd.DataFrame(scaler.fit_transform(data),index=eduGrouped.index,columns=['score_scaler']) # 점수는 정규화

In [48]:
eduGrouped=eduGrouped.drop(['hischool_type_seq', 'hischool_special_type',
       'hischool_nation', 'hischool_gender', 'hischool_location_seq',
       'univ_type_seq1', 'univ_type_seq2', 'univ_transfer', 'univ_location',
       'univ_major', 'univ_sub_major', 'univ_major_type'],axis=1)

In [49]:
eduGrouped = pd.concat([eduGrouped,data2],axis=1)
eduGrouped=eduGrouped.drop(['univ_score'],axis=1)

In [50]:
eduGrouped

,hischool_type_0,hischool_type_2,hischool_type_8,hischool_type_10,hischool_type_12,hischool_type_16,hischool_type_18,hischool_type_19,hischool_type_20,hischool_type_21,...,major_type_15,major_type_16,major_type_17,major_type_18,major_type_19,major_type_20,major_type_21,major_type_22,major_type_23,score_scaler
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.2
U00002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.9
U00003,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.9
U00004,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.7
U00005,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0.0
U08479,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
U08480,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.8
